In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil as sh
import random as rng
import glob
# import itertools

import librosa as lb
from librosa.display import specshow
import IPython.display as ipd


import sklearn as sk
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#  --  Defining Variables  --  #

max_ms = 4000

batchs = 128
epochs = 100

ind_to_label = {
    0 : 'angry',
    1 : 'fear',
    2 : 'happy',
    3 : 'neutral',
    4 : 'sad'
}

label_to_ind = { 
    lab: ind for ind, lab in ind_to_label.items()
    }

In [ ]:
'''
Data insights

'''

class aud_stats:
    @staticmethod
    def average_sr():
        pass


In [ ]:
'''

PREPROCESSING UTILS


'''


class aud_util:
    
    @staticmethod
    def loadaud(audio_file_path, sr=None, mono=False):                                 # load audio file, *mono argument (bool) can auto convert to mono, while default sr is converted to 22050*
        return lb.load(audio_file_path, sr=sr, mono=mono)                              # returns (data, sr)       

    @staticmethod
    def pad_trunc(aud, sr, target_ms):                                                 # padding places shorter audio randomly within the time frame of the padded length
        maxlen = (target_ms//1000)*sr
        
        if len(aud) == maxlen:
            return aud, sr

        elif len(aud) > maxlen:
            return aud[:maxlen], sr

        elif len(aud) < maxlen:
            pad = maxlen - len(aud)
            pad = np.zeros((pad))
            return np.concatenate((aud, pad), 0), sr



class aud_img:
    @staticmethod
    def melspec(data, sr):                                                             # returns 3 channels, deplicated from 1
        spec = lb.feature.melspectrogram(data, sr=sr, power=1)                         # power = 1/2 changes amplitude_to_db or power_to_db
        spec = lb.amplitude_to_db(spec, ref=np.min)
        spec = np.expand_dims(spec, axis=2)
        return np.stack((spec,)*3, axis=2).squeeze()

    @staticmethod
    def mfcc(data, sr):                                                                # returns 3 channels, deplicated from 1
        mfcc_ = lb.feature.mfcc(data, sr)
        #mfcc_ = sk.preprocessing.scale(mfcc_, axis=1)
        mfcc_ = np.expand_dims(mfcc_, axis=2)
        return np.stack((mfcc_,)*3, axis=2).squeeze()

    @staticmethod
    def display_audio_img(spec, sr , mfcc=False):
        fig, ax = plt.subplots()
        
        if mfcc:
            specshow(spec, sr=sr, x_axis='time')
        else:
            img = specshow(spec, x_axis='time', y_axis='mel', sr=sr, fmax=8000, ax=ax)
            fig.colorbar(img, ax=ax, format='%+2.0f dB')


class rav_prep:
    '''
    01 = neutral, 
    02 = calm,  -
    03 = happy, 
    04 = sad, 
    05 = angry, 
    06 = fearful, 
    07 = disgust,  -
    08 = surprised -
    '''
    @staticmethod
    def correct_data_type(path):
        if (path.split('/')[-1].split('-')[0] == '03') and (path.split('/')[-1].split('-')[1] == '01') and (path.split('/')[-1].split('-')[2] in ['01', '03', '04', '05', '06']):
          return True
        else:
          return False
    
    @staticmethod
    def filter(path):
      counter = 0
      for i in glob.glob(path):
        if rav_prep.correct_data_type(i):
          continue
        elif rav_prep.correct_data_type(i) != True:
          sh.move(i, '/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/A_removed_files')
          counter += 1
          continue
      print(f'removed {counter} files')

    @staticmethod
    def move_ravdess_colab(path):                                               # colab google drive paths
      for i in glob.glob(path):
        if i.split('/')[-1].split('-')[2] == '05':
          sh.copy(i, '/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/angry')
        
        elif i.split('/')[-1].split('-')[2] == '06':
          sh.copy(i, '/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/fear')

        elif i.split('/')[-1].split('-')[2] == '03':
          sh.copy(i, '/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/happy')

        elif i.split('/')[-1].split('-')[2] == '01':
          sh.copy(i, '/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/neutral')

        elif i.split('/')[-1].split('-')[2] == '04':
          sh.copy(i, '/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/sad')


class ds_create:
    
    @staticmethod    
    def label_from_bpath(bpath):                                                       # probably will not be used
        return bpath.decode('utf-8').split('\\')[-2]

    @staticmethod
    def slices_for_onelabel(path, label):                                              
        paths = glob.glob(path)

        labels = [label_to_ind[label]]*len(paths)

        return paths , labels

    @staticmethod
    def dfpremel(path):
        data, sr = aud_util.loadaud(path, sr=16000, mono=True)
        data, sr = aud_util.pad_trunc(data, sr, max_ms)                                
        mel = aud_img.melspec(data, sr)
        return mel
    
    @staticmethod
    def dfpremfcc(path):
        data, sr = aud_util.loadaud(path, sr=16000, mono=True)
        data, sr = aud_util.pad_trunc(data, sr, max_ms)                                
        mel = aud_img.mfcc(data, sr)
        return mel



In [ ]:
'''
organising ravdess data 
Done once only, by the time you see this cell, it probably was already run, so you can ignore it 
as all the revdess files have already been organised into the sub-emotion folder in the google drive, in
/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data

'''

rav_prep.filter('/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/Altogether/*.wav')
rav_prep.move_ravdess_colab('/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/Altogether/*.wav')


removed 576 files


In [ ]:
'''
BUILDING DATASET PIPELINE (only original training data included)

'_o' means original data, excluding any extra data

'''



angry_o, _0 =   ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/NLP Training Dataset/ASR Training Dataset/angry/*.wav', 'angry')
fear_o, _1 =    ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/NLP Training Dataset/ASR Training Dataset/fear/*.wav', 'fear')
happy_o, _2 =   ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/NLP Training Dataset/ASR Training Dataset/happy/*.wav', 'happy')
neutral_o, _3 = ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/NLP Training Dataset/ASR Training Dataset/neutral/*.wav', 'neutral')
sad_o, _4 =     ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/NLP Training Dataset/ASR Training Dataset/sad/*.wav', 'sad')

slices = angry_o + fear_o + happy_o + neutral_o + sad_o
labels = _0 + _1 + _2 + _3 + _4


In [ ]:
'''
BUILDING DATASET PIPELINE (both original data and ravdess)

'_o' means original data, excluding any extra data

'''

angry_o, _0 =   ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/NLP Training Dataset/ASR Training Dataset/angry/*.wav', 'angry')
fear_o, _1 =    ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/NLP Training Dataset/ASR Training Dataset/fear/*.wav', 'fear')
happy_o, _2 =   ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/NLP Training Dataset/ASR Training Dataset/happy/*.wav', 'happy')
neutral_o, _3 = ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/NLP Training Dataset/ASR Training Dataset/neutral/*.wav', 'neutral')
sad_o, _4 =     ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/NLP Training Dataset/ASR Training Dataset/sad/*.wav', 'sad')

angry_r, r_0 =   ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/angry/*.wav', 'angry')
fear_r, r_1 =    ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/fear/*.wav', 'fear')
happy_r, r_2 =   ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/happy/*.wav', 'happy')
neutral_r, r_3 = ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/neutral/*.wav', 'neutral')
sad_r, r_4 =     ds_create.slices_for_onelabel(r'/content/drive/MyDrive/NLP/RAVDESS_altogether/modified_data/sad/*.wav', 'sad')


slices = angry_o + fear_o + happy_o + neutral_o + sad_o    +   angry_r + fear_r + happy_r + neutral_r + sad_r
labels = _0 + _1 + _2 + _3 + _4   +   r_0 + r_1 + r_2 + r_3 + r_4



In [ ]:
'''
Dataframe style

using tf.stack later lol
'''


df = pd.DataFrame()

df['relative_audio_paths'] = slices
df['int_labels'] = labels
df['1hot_labels'] = list(to_categorical(labels))

df['imgs_3c'] = list(map(ds_create.dfpremel, slices))


df = sk.utils.shuffle(df)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.head()

,relative_audio_paths,int_labels,1hot_labels,imgs_3c
0,/content/drive/MyDrive/NLP/RAVDESS_altogether/...,1,"[0.0, 1.0, 0.0, 0.0, 0.0]","[[[37.347054, 37.347054, 37.347054], [37.34705..."
1,/content/drive/MyDrive/NLP/NLP Training Datase...,2,"[0.0, 0.0, 1.0, 0.0, 0.0]","[[[63.72838126616919, 63.72838126616919, 63.72..."
2,/content/drive/MyDrive/NLP/NLP Training Datase...,4,"[0.0, 0.0, 0.0, 0.0, 1.0]","[[[87.36988820126425, 87.36988820126425, 87.36..."
3,/content/drive/MyDrive/NLP/RAVDESS_altogether/...,1,"[0.0, 1.0, 0.0, 0.0, 0.0]","[[[38.92079559238242, 38.92079559238242, 38.92..."
4,/content/drive/MyDrive/NLP/NLP Training Datase...,4,"[0.0, 0.0, 0.0, 0.0, 1.0]","[[[76.25816571916035, 76.25816571916035, 76.25..."


In [ ]:
df.iloc[0,3].shape

(128, 126, 3)

In [ ]:
'''
ignore this cell

'''
# requires shuffling in tf.keras.Model.fit

X = list(map(ds_create.path_to_mel, slices))

Y = np.array(to_categorical(labels))
X = np.array(X)


#ds = ds.map(ds_create.preprocess_mel_eachlabel)
ds = ds.map(ds_create.preprocess_mel_eachlabel)
# ds = ds.cache()
# ds = ds.batch(batchs)
# ds = ds.prefetch(16)

In [ ]:
input_shape = df.iloc[0,3].shape
input_shape

(128, 126, 3)

In [ ]:
xin = Input(input_shape)

prenet = tf.keras.applications.efficientnet_v2.EfficientNetV2S(weights='imagenet', include_top=False)#, input_shape=input_shape)
x = prenet(xin)

x = Flatten()(x)
x = Dense(256, activation='swish')(x)
x = Dropout(0.65)(x)
x = Dense(128, activation='swish')(x)
x = Dropout(0.5)(x)


xout = Dense(5, activation='softmax')(x)

own = Model(xin, xout)
own.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc', tfa.metrics.F1Score(num_classes=5, average='weighted', threshold=0.5)])
own.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 126, 3)]     0         
                                                                 
 efficientnetv2-s (Functiona  (None, None, None, 1280)  20331360 
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 20480)             0         
                                                                 
 dense (Dense)               (None, 256)               5243136   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                             

In [ ]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.1, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(r'/content/drive/MyDrive/NLP/Zhihaos stuff/effinet v2s', monitor='val_loss', verbose=0, save_best_only=True)
]

In [ ]:
history = own.fit(
    x=tf.stack(df['imgs_3c']),
    y=tf.stack(df['1hot_labels']),
    batch_size=batchs,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
)

Epoch 1/100
28/28 [==============================] - 130s 4s/step - loss: 1.5855 - acc: 0.3710 - f1_score: 0.2792 - val_loss: 1.1825 - val_acc: 0.5464 - val_f1_score: 0.3523 - lr: 0.0010
Epoch 2/100
28/28 [==============================] - 10s 367ms/step - loss: 1.0994 - acc: 0.5783 - f1_score: 0.5257 - val_loss: 1.1956 - val_acc: 0.6025 - val_f1_score: 0.5603 - lr: 0.0010
Epoch 3/100
28/28 [==============================] - 107s 4s/step - loss: 0.8599 - acc: 0.6772 - f1_score: 0.6622 - val_loss: 0.7853 - val_acc: 0.7251 - val_f1_score: 0.7014 - lr: 0.0010
Epoch 4/100
28/28 [==============================] - 10s 368ms/step - loss: 0.8243 - acc: 0.7178 - f1_score: 0.6932 - val_loss: 0.8167 - val_acc: 0.6930 - val_f1_score: 0.6355 - lr: 0.0010
Epoch 5/100
28/28 [==============================] - 10s 367ms/step - loss: 0.7875 - acc: 0.7210 - f1_score: 0.6983 - val_loss: 0.8913 - val_acc: 0.6930 - val_f1_score: 0.6783 - lr: 0.0010
Epoch 6/100
28/28 [==============================] - 106s 4

In [ ]:
tst = ds_create.dfpremel(r'/content/drive/MyDrive/NLP/NLP Training Dataset/ASR Training Dataset/fear/00530e07e3.wav')
tst = ds_create.dup_channel(tst)
tst = np.expand_dims(tst, axis=0)                                             # EXPAND DIMS OF FIRST DIMENSION ARGHHHHHH
pred = own.predict(tst)
pred = np.argmax(pred)
pred

In [ ]:
'''
Generating the qualifying csv file

'''


class test_gen:
    @staticmethod
    def path_gen(path):                                              #for zhihao's local pc
        paths = os.listdir(path)
        paths = list(map(lambda x : 'DATA_NLP_TIL\\.qualifying_test\\'+x , paths))

        return paths 

    @staticmethod
    def path_gen_colab(path):                                              #for colab, zhihaos
        paths = os.listdir(path)
        paths = list(map(lambda x : '/content/drive/MyDrive/NLP/NLP Interim Dataset/NLP/'+x , paths))

        return paths 

    @staticmethod
    def path_to_mel(path):
        ccc = ds_create.dfpremel(path)
        return ccc
    
    @staticmethod
    def path_to_mfcc(path):
        ccc = ds_create.dfpremfcc(path)
        return ccc

    @staticmethod
    def int_to_label(int):
        return ind_to_label[int]
        

In [ ]:
q_df = pd.DataFrame()
paths = sorted(glob.glob(r'/content/drive/MyDrive/NLP/NLP Interim Dataset/NLP/*.wav'))
q_data = list(map(test_gen.path_to_mel, paths))

q_data = tf.stack(q_data)

preds = own.predict(q_data)
preds = np.argmax(preds, axis=1)

In [ ]:
q_df['paths'] = sorted(os.listdir(r'/content/drive/MyDrive/NLP/NLP Interim Dataset/NLP/'))
q_df['labels'] = list(map(
    test_gen.int_to_label,
    list(preds)
))

In [ ]:
q_df.head()

,paths,labels
0,00ae09ba94.wav,angry
1,00f2a00f1f.wav,angry
2,012822b908.wav,fear
3,0144091c26.wav,sad
4,0145cb0279.wav,sad


In [ ]:
q_df.to_csv(r'/content/drive/MyDrive/NLP/Zhihao nlp preds/qualifiers4.csv', header=False, index=False)